In [1]:
# Import the necessary modules
import json
import requests
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from rich import print as rprint
import pandas as pd

In [ ]:
# Wärmepreisindex
cubecode = '61111BM006'
areatype = 'all'
category = 'all'
content = ''
start_year = ''
classifier1 = 'CC13B1'
key1 = 'CC13-77'
lang = 'en'

# Destatis Base Url
url = (f"{genesis_url}username={user_id}&password={password}&language=de&name={cubecode}&area={areatype}"
       f"&compress=true&contents={content}&startyear={start_year}")

# Add Classifyers to url
if classifier1:
       url += f'&classifyingvariable1={classifier1}'    
if key1:
       url += f'&classifyingkey1={key1}'

# request 
response = requests.get(url)
data = json.loads(response.text)["Object"]['Content'].split('\n')

# Filter out the relevant lines
filtered_response = [line for line in data if 'D' in line and 'e' in line and ('MONAT' in line or 'QUART' in line)]

# Split each line into its components and extract the relevant information
data = [{'field_D': parts[0],
         'field_DG': parts[1],
         'classifyingkey1': parts[2],
         'period': parts[3],
         'year': int(parts[4]),
         'value': float(parts[5]),
         'field_e': parts[6]
        }
        for parts in (line.split(';') for line in filtered_response)]

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Determine if the data is monthly or quarterly based on the unique values in the 'period' column
if df['period'].str.contains('Q').any():
    period_key = 'quarter'
else:
    period_key = 'month'

# Create result dict
result = {'cubeCode': cubecode, 'content': content, 'classifyingVar1': classifier1, 'classifyingKey1': key1, 'data': []}

for year, group in df.groupby('year'):
    year_data = {'year': year, 'df': [{period_key: row['period'], 'value': row['value']} for _, row in group.iterrows()]}
    result['data'].append(year_data)
    
rprint(json.dumps(result, indent=4))